In [ ]:
import sys
import os
import numpy as np
import matplotlib.pylab as plt
import itertools
import torch

sys.path.append("..")
sys.path.append("../framework")

In [ ]:
from history import History
from loader import Loader
from util import drawColorMapOfEventProb

In [ ]:
from tppWithPv.agentTpp import AgentFactoryTpp
from tppWithPv.environmentTpp import EnvironmentFactoryTpp
from tppWithPv.trainerTpp import TrainerFactoryTpp

from tppWithPv.agentTpp import AgentHawkesWithPv
from tppWithPv.environmentTpp import EventDataFeederWithPv
from tppWithPv.trainerTpp import TrainerTppMLE

## S100: Define the trainer ids to load

In [ ]:
class LoaderTpp(Loader):

    def iterateTrainId(self):
        for trainerId in (1,2,3):
            yield trainerId

## S200: Analyze the result of trainings

### SS210: define a loader of the trio (agent, environment, trainer) 

In [ ]:
history = History("historyTpp.sqlite")
agentFactoryTpp = AgentFactoryTpp()
environmentFactoryTpp = EnvironmentFactoryTpp()
trainerFactoryTpp = TrainerFactoryTpp()
loader = LoaderTpp(history, agentFactoryTpp, environmentFactoryTpp, \
    trainerFactoryTpp)

### SS220: check the learning curves for each training

In [ ]:
def plotLearningCurve(loader):

    fig = plt.gcf()
    ax = fig.gca()
    for k1, (agent, environment, trainer) in enumerate(loader.iterateHistory()):
        timestamp, tbl = trainer.getTrainLog()
        epoch, ll = zip(*tbl)
        ax.plot(epoch, ll, label = "#%d" % (k1 + 1))
    ax.set_xlabel('Epcoh')
    ax.set_ylabel('LogLikelihood')
    ax.legend()

In [ ]:
fig = plt.figure()
plotLearningCurve(loader)
plt.tight_layout()
plt.legend()
pass

### SS230: check the estimation results for each model

In [ ]:
def plotEstimationOnPatchColor(agent, environment, idxSelected):

    Nseq = environment.Nseq
    Ndelta = environment.eventDataSet.getNdelta()
    Npv = environment.pvDataSet.getNpv()

    E, Pv = environment.getTrainData()
    _E = torch.tensor(E[:, idxSelected, None, :]) # (Nseq, 1, Ndelta)
    _Pv = torch.tensor(Pv[:, idxSelected, None, :]) # (Nseq, 1, Npv)

    _I = agent(_E, _Pv)

    I = _I.data.numpy() # (Nseq+1, 1, Ndelta)

    xTickLabel = ["" for _ in range(Nseq)]
    yDeltaTickLabel = ["Delta #%d" % k1 for k1 in range(Ndelta)]
    yPvTickLabel = ["Pv #%d" % k1 for k1 in range(Npv)]

    F_NULL = np.zeros((Nseq, Npv)) # (Nseq, Npv)

    fig = plt.gcf()
    #
    ax = fig.add_subplot(2,1,1)
    drawColorMapOfEventProb(E[:,idxSelected,:], I[:-1,0,:], xTickLabel, yDeltaTickLabel, None, cmap = "Reds")
    #
    ax = fig.add_subplot(2,1,2)
    drawColorMapOfEventProb(F_NULL, Pv[:,idxSelected,:], xTickLabel, yPvTickLabel, None, cmap = "Blues")

In [ ]:
for k1, (agent, environment, trainer) in enumerate(loader.iterateHistory()):    
    idxSelected = np.random.choice(environment.getAvailableIndex())
    fig = plt.figure()
    plotEstimationOnPatchColor(agent, environment, idxSelected)
    plt.tight_layout()